In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3,4,5,6'

import tensorflow as tf
tf.config.list_physical_devices('GPU')

2025-04-10 15:18:46.129337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744278526.149800 3010976 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744278526.156191 3010976 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744278526.171170 3010976 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744278526.171189 3010976 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744278526.171191 3010976 computation_placer.cc:177] computation placer alr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU')]

In [2]:
import datasets

dataset = datasets.load_dataset("coastalcph/lex_glue", "scotus")

import math
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import DatasetDict

# ----- Step 1: Load the Summarization Model -----
# We use a long-document summarization model that can process lengthy texts.
sum_model_name = "nsi319/legal-led-base-16384"
sum_tokenizer = AutoTokenizer.from_pretrained(sum_model_name)
sum_model = AutoModelForSeq2SeqLM.from_pretrained(sum_model_name)

# Create a summarization pipeline (you can adjust parameters later)
summarizer = pipeline("summarization", model=sum_model, tokenizer=sum_tokenizer, framework="pt")

# ----- Step 2: Define a Function to Summarize a Document -----
def summarize_document(document, summarizer, tokenizer, chunk_size=15000, final_max_length=512):
    """
    Summarize a long document in two stages:
      - If document is longer than chunk_size tokens, split into chunks,
        summarize each chunk, then combine and summarize again.
      - Otherwise, summarize directly.
    
    Args:
        document (str): The full text of the document.
        summarizer: The Hugging Face summarization pipeline.
        tokenizer: The tokenizer corresponding to the summarization model.
        chunk_size (int): Maximum tokens per chunk.
        final_max_length (int): Maximum token length for final summary.
    
    Returns:
        str: Final summary text.
    """
    # Tokenize document without truncation
    tokens = tokenizer.encode(document, truncation=False)
    num_tokens = len(tokens)
    
    # If the document is within chunk_size, summarize directly.
    if num_tokens <= chunk_size:
        summary = summarizer(document, max_length=final_max_length, truncation=True)[0]['summary_text']
        return summary
    else:
        num_chunks = math.ceil(num_tokens / chunk_size)
        chunk_summaries = []
        
        # Process each chunk
        for i in range(num_chunks):
            start = i * chunk_size
            end = min((i + 1) * chunk_size, num_tokens)
            # Decode the token IDs for the current chunk back into text.
            chunk_token_ids = tokens[start:end]
            chunk_text = tokenizer.decode(chunk_token_ids, skip_special_tokens=True)
            # Summarize the chunk (you can adjust max_length for intermediate summaries)
            chunk_summary = summarizer(chunk_text, max_length=final_max_length, truncation=True)[0]['summary_text']
            chunk_summaries.append(chunk_summary)
        
        # Combine all chunk summaries into one text.
        combined_summary_text = " ".join(chunk_summaries)
        # Optionally, if the combined summary is still long, summarize it one more time.
        final_summary = summarizer(combined_summary_text, max_length=final_max_length, truncation=True)[0]['summary_text']
        return final_summary

# ----- Step 3: Apply Summarization to the Training Split Only -----
def summarize_example(example):
    # Assume each example has a "text" field with the long legal document.
    example["text"] = summarize_document(example["text"], summarizer, sum_tokenizer,
                                          chunk_size=15000, final_max_length=400)
    return example

# If your dataset is a Hugging Face DatasetDict (with splits 'train', 'test', 'validation'):
# We only summarize the training set and leave the others as is.
summarized_dataset = DatasetDict({
    "train": dataset["train"],
    "test": dataset["test"],
    "validation": dataset["validation"].map(summarize_example)
})

Device set to use cuda:0


Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

Input ids are automatically padded from 8559 to 9216 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 3877 to 4096 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 10193 to 10240 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 12256 to 12288 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 11132 to 11264 to be a multiple of `config.attention_window`: 1024
Token indices sequence length is longer than the specified maximum sequence length for this model (18651 > 16384). Running this sequence through the model will result in indexing errors
Input ids are automatically padded from 15001 to 15360 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 3652 to 4096 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 798 to 1024 to be a multiple of

In [3]:
summarized_dataset.push_to_hub("victorambrose11/summarized_scotus_latest_validation")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/victorambrose11/summarized_scotus_latest_validation/commit/f7c17bc9234b4715fc46d90c593b5d1b3d67b640', commit_message='Upload dataset', commit_description='', oid='f7c17bc9234b4715fc46d90c593b5d1b3d67b640', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/victorambrose11/summarized_scotus_latest_validation', endpoint='https://huggingface.co', repo_type='dataset', repo_id='victorambrose11/summarized_scotus_latest_validation'), pr_revision=None, pr_num=None)